### **Data Loading, Storage, and File Formats**

#### **6.1 Reading and Writing Data in Text Format**

##### ***Mechanics of the function***

1. ***Indexing*** -- Can treat one or more columns as the returned Dataframe, and whether to get columns names from the files, arguments you provide or not at all.
2. ***Type inference and data conversion*** -- the user defined value conversions and custom list of missing value markers.
3. ***Data and Time parsing*** -- Includes combining capability, including combining date and time information spread over multiple columns into a single columns in a result.
4. ***Iterating*** -- Support for iterating over chunks of very large files.
5. ***Unclean data issue*** -- Skipping rows or a footer, comments, or other minor things like numeric data with thousands seperated with commas.


In [1]:
import pandas as pd 
import numpy as np

In [2]:
!type "..\data\ex1.csv"

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [3]:
# since it is comma-delimited, we can read_csv
df = pd.read_csv("../data/ex1.csv")

In [4]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# a file will not always have a header row.
!type "..\data\ex2.csv"

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [6]:
# options-  1. you can allow pandas to assign a default comlumn name, or you can specify names yourself
pd.read_csv('../data/ex2.csv',header = None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv('../data/ex2.csv',names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
# suppose you wanted the message column to be the index of the returned DataFrame.
names = ['a','b','c','d','message']
pd.read_csv('../data/ex2.csv',names = names,index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [9]:
# in the event you want to form a heirarchical index from multiple column, pass a list of column numbers or names:
!type ..\data\csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [10]:
parsed = pd.read_csv('../data/csv_mindex.csv',index_col=['key1','key2'])

In [11]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [12]:
# in some cases, a table might not have a fixed delimiter, using a whitespace or some other pattern to seperate fields
list(open('../data/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [13]:
result = pd.read_csv('../data/ex3.txt',sep= r'\s+')

In [14]:
result # here since the number of columns is less than the number of rows, read_csv infers that the first column must be the index in this special case. 

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [15]:
# exploring the additional arguments in the parser function
!type ..\data\ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [16]:
pd.read_csv('../data/ex4.csv',skiprows=[0,2,3]) # skips these rows

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [17]:
!type ..\data\ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [18]:
result = pd.read_csv('../data/ex5.csv')

In [19]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [21]:
result = pd.read_csv('../data/ex5.csv',na_values=['NULL']) # to consider null values

In [22]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [23]:
# different NA sentinels can be specified for each column in a dict:
sentinels = {'message':['foo','NA'],'something':['two']}
pd.read_csv('../data/ex5.csv',na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


#### ***Reading Text files in Pieces*** 

In [24]:
pd.options.display.max_rows=10

In [25]:
result = pd.read_csv('../data/ex6.csv')

In [26]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [27]:
# if you want a small number of rows
pd.read_csv('../data/ex6.csv',nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [28]:
# you can also specify the chunk size
chunker = pd.read_csv('../data/ex6.csv',chunksize=1000)

In [29]:
chunker 
# the TextFileReader object returned allows you to iterate over the parts of file according to the chunksize.

In [30]:
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value=0)
tot.sort_values(ascending=False)
# TextFilereader is equipped with get_chunk that enales you to read pieces of an arbitrary size

key
E    368
X    364
L    346
O    343
Q    340
    ... 
5    157
2    152
0    151
9    150
1    146
Length: 36, dtype: object

#### ***Writing Data to Text Format*** 

In [31]:
# Data can also be exported to delimited format
data = pd.read_csv('../data/ex5.csv')

In [32]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [33]:
data.to_csv('../data/out.csv')

In [34]:
!type ..\data\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [35]:
import sys

In [36]:
data.to_csv(sys.stdout,sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [37]:
# missing values appear a empty strings. you might want to change them to something more visible
data.to_csv(sys.stdout,sep='|',na_rep='NULL')

|something|a|b|c|d|message
0|one|1|2|3.0|4|NULL
1|two|5|6|NULL|8|world
2|three|9|10|11.0|12|foo


In [38]:
# row and column labels can be disabled
data.to_csv(sys.stdout,index=False,header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [39]:
# write only subset of column
data.to_csv(sys.stdout,index=False,columns=['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [40]:
# Series also has to_csv method
index = np.arange(10,17)
series = pd.Series(np.arange(7),index=index)
series.to_csv('../data/series.csv')

In [41]:
!type ..\data\series.csv

,0
10,0
11,1
12,2
13,3
14,4
15,5
16,6
